In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
import math

import time

In [2]:
# Importando .csv
train = pd.read_csv('dataset/train_data.csv', header=None,  skiprows=[0], usecols=[0,1,2])
m = pd.read_csv('dataset/movies_data.csv')

# Obtendo generos dos filmes
m = pd.read_csv('dataset/movies_data.csv')
genres = pd.get_dummies(m.set_index(['movie_id']).genres.str.split('|', expand=True).stack(dropna=False)).sum(level=0)

In [3]:
# Função FunkSVD
def funkSVD(train, k = 5, lr = 0.05, reg = 0.02, miter = 10):
    global_mean = train[2].mean(skipna = True)
    nusers = train[0].max()+1
    nitems = train[1].max()+1
    bu = np.full(nusers, 0, dtype=float)
    bi = np.full(nusers, 0, dtype=float)
    # P = np.full((nusers+1, k), 0.1)
    # Q = np.full((nitems+1, k), 0.1)
    P = np.random.normal(loc = 0, scale = 0.1, size=(nusers, k))
    Q = np.random.normal(loc = 0, scale = 0.1, size=(nitems, k))
    for f in range(k):
        for l in range(0, miter):
            for j in train.index:
                u = train[0][j]
                i = train[1][j]
                r_ui = train[2][j]
                pred = global_mean + bu[u] + bi[i] + np.dot(P[u, ], Q[i, ])
                e_ui = r_ui - pred
                bu[u] = bu[u] + lr * e_ui # - reg * bu[u]
                bi[i] = bi[i] + lr * e_ui # - reg * bi[i]
                temp_uf = P[u, f]
                P[u, f] = P[u, f] + lr * (e_ui * Q[i, f] - reg * P[u, f])
                Q[i, f] = Q[i, f] + lr * (e_ui * temp_uf - reg * Q[i, f])

    return { "global_mean": global_mean, "bu": bu, "bi": bi, "P": P, "Q": Q }

In [4]:
# Predizendo nota
def predict(model, u, i):
    return model["global_mean"] + model["bu"][u] + model["bi"][i] + np.dot(model["P"][u], model["Q"][i])

In [5]:
# Avaliando predições
def rmse(model, test):
    sum_err = 0
    for t in test:
        u = t[0]
        i = t[1]
        r_ui = t[2]
        pred = predict(model, u, i)
        error = (r_ui - pred)**2
        sum_err += error
    return math.sqrt(sum_err/len(test))

In [6]:
def validation(model, data, train_split = 0.75, k = 5, lr = 0.05, reg = 0.02, miter = 10):
    data = data.sample(frac=1)
    train_len = int(train_split * len(data))
    train = data[:train_len]
    test = data[train_len:].values

    start_time = time.time()
    model = model(train, k, lr, reg, miter)
    print("Tempo de treinamento em segundos: ", time.time() - start_time)
    start_time = time.time()
    print("RMSE :", rmse(model, test))
    print("Tempo de predição em segundos: ", time.time() - start_time)

In [7]:
validation(funkSVD, train, 0.75, k=9)

Tempo de treinamento em segundos:  1384.3588817119598
RMSE : 0.8917648049441322
Tempo de predição em segundos:  1.4675226211547852



## Execução 1 treino 75% k = 2, lr = 0.07, reg = 0.02, miter = 1
Tempo de treinamento em segundos:  44.2860951423645 <br >
RMSE : 0.9257016292902414 <br >
Tempo de predição em segundos:  0.9218599796295166 <br >

## Execução 2 treino 75% k = 2, lr = 0.07, reg = 0.02, miter = 10
Tempo de treinamento em segundos:  523.6071541309357 <br >
RMSE : 0.8962728810026056 <br >
Tempo de predição em segundos:  1.095106840133667 <br >

## Execução 3 treino 75% k = 5, lr = 0.07, reg = 0.02, miter = 10
Tempo de treinamento em segundos:  1384.3588817119598 <br >
RMSE : 0.8917648049441322 <br >
Tempo de predição em segundos:  1.4675226211547852 <br >